Сегодня мы поговорим о морфологических анализаторах для русского языка. Важно отметить, что (в основном) для каждого естественного языка нужен свой морфоанализатор.

* шаг 1: установим пайморфи и майстем

In [ ]:
# !pip3 install pymystem3==0.1.10
# !pip3 install pymorphy2[fast]

# ! pip3 install gensim

In [22]:
# импортируем нужные части библиотек
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

# сохраняем класс в переменную
mystem = Mystem()
morph = MorphAnalyzer()
print("Hello")


Installing mystem to /Users/azhebel/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


Hello


In [5]:
# сэмпл-текст, на котором все будем пробовать
text = """Зрелый фрукт на вкус очень сладок и обладает приятным сладковатым ароматом. В нём много витаминов и сахаров, но мало кислот."""

## нормализация

давайте приведем текст к нижнему регистру

In [15]:
text = text.lower()
text

'зрелый фрукт на вкус очень сладок и обладает приятным сладковатым ароматом. в нём много витаминов и сахаров, но мало кислот.'

## токенизация

иногда текст нужно предварительно разбить на токены, отдельные элементы. Токенизаторов много, каждый делает это, руководствуясь своими правилами

* токенизация нужна, если морофоанализатор не умеет токенизировать сам (Mystem умеет, Pymorphy не умеет)

In [20]:
# два примера токенизации
from nltk.tokenize import word_tokenize
from gensim.utils import tokenize

In [21]:
list(tokenize(text))

['зрелый',
 'фрукт',
 'на',
 'вкус',
 'очень',
 'сладок',
 'и',
 'обладает',
 'приятным',
 'сладковатым',
 'ароматом',
 'в',
 'нём',
 'много',
 'витаминов',
 'и',
 'сахаров',
 'но',
 'мало',
 'кислот']

In [23]:
word_tokenize(text)

['зрелый',
 'фрукт',
 'на',
 'вкус',
 'очень',
 'сладок',
 'и',
 'обладает',
 'приятным',
 'сладковатым',
 'ароматом',
 '.',
 'в',
 'нём',
 'много',
 'витаминов',
 'и',
 'сахаров',
 ',',
 'но',
 'мало',
 'кислот',
 '.']

## лемматизация и морфоанализ

### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно ему подавать сырой текст.

In [6]:
# сначала лемматизируем слова методом .lemmatize()
print(text, "\n")

print(mystem.lemmatize(text))


зрелый фрукт на вкус очень сладок и обладает приятным сладковатым ароматом. в нём много витаминов и сахаров, но мало кислот. 

['зрелый', ' ', 'фрукт', ' ', 'на', ' ', 'вкус', ' ', 'очень', ' ', 'сладкий', ' ', 'и', ' ', 'обладать', ' ', 'приятный', ' ', 'сладковатый', ' ', 'аромат', '.', ' ', 'в', ' ', 'он', ' ', 'много', ' ', 'витамин', ' ', 'и', ' ', 'сахаров', ', ', 'но', ' ', 'мало', ' ', 'кислота', '.', '\n']


In [24]:
# метод .analyze() даст грамматическую информацию о словах

ms_analyzed = mystem.analyze(text)

этот метод возвращает список словарей


каждый словарь имеет либо одно поле 'text' (когда попался пробел или пунктуация) или text и analysis

* в analysis снова список словарей с вариантами разбора (первый самый вероятный)
* поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
* analysis - может быть пустым списком

In [25]:
print(ms_analyzed) # попробуйте разные индексы

[{'analysis': [{'lex': 'зрелый', 'wt': 1, 'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}], 'text': 'зрелый'}, {'text': ' '}, {'analysis': [{'lex': 'фрукт', 'wt': 0.9485761648, 'gr': 'S,муж,неод=(вин,ед|им,ед)'}], 'text': 'фрукт'}, {'text': ' '}, {'analysis': [{'lex': 'на', 'wt': 0.9989522965, 'gr': 'PR='}], 'text': 'на'}, {'text': ' '}, {'analysis': [{'lex': 'вкус', 'wt': 1, 'gr': 'S,муж,неод=(вин,ед|им,ед)'}], 'text': 'вкус'}, {'text': ' '}, {'analysis': [{'lex': 'очень', 'wt': 1, 'gr': 'ADV='}], 'text': 'очень'}, {'text': ' '}, {'analysis': [{'lex': 'сладкий', 'wt': 1, 'gr': 'A=ед,кр,муж'}], 'text': 'сладок'}, {'text': ' '}, {'analysis': [{'lex': 'и', 'wt': 0.9999770357, 'gr': 'CONJ='}], 'text': 'и'}, {'text': ' '}, {'analysis': [{'lex': 'обладать', 'wt': 1, 'gr': 'V,несов,нп=непрош,ед,изъяв,3-л'}], 'text': 'обладает'}, {'text': ' '}, {'analysis': [{'lex': 'приятный', 'wt': 1, 'gr': 'A=(дат,мн,полн|твор,ед,полн,муж|твор,ед,полн,сред)'}], 'text': 'приятным'}, {'text': ' '}, {'analys

In [26]:
# сделаем все красиво с индексами и доступом по ключам

print('Слово - ',ms_analyzed[0]['text'])
print('Разбор слова - ', ms_analyzed[0]['analysis'][0])
print('Лемма слова - ', ms_analyzed[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', ms_analyzed[0]['analysis'][0]['gr'])

Слово -  зрелый
Разбор слова -  {'lex': 'зрелый', 'wt': 1, 'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}
Лемма слова -  зрелый
Грамматическая информация слова -  A=(вин,ед,полн,муж,неод|им,ед,полн,муж)


In [27]:
# леммы можно достать в одну строчку
[elem['analysis'][0]['lex'] for elem in ms_analyzed if elem.get('analysis')]

['зрелый',
 'фрукт',
 'на',
 'вкус',
 'очень',
 'сладкий',
 'и',
 'обладать',
 'приятный',
 'сладковатый',
 'аромат',
 'в',
 'он',
 'много',
 'витамин',
 'и',
 'сахаров',
 'но',
 'мало',
 'кислота']

In [ ]:
# # то же самое, что в предыдущей ячейке, но циклом
# res = []
# for elem in ms_analyzed:
#     if elem.get('analysis'):
#         res.append(elem['analysis'][0]['lex'])

# res[:10]

#### дополнительные возможности Mystem

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/ и использовать в командной строке

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

### Pymorphy

Pymorphy - открытый и развивается ([можно поучаствовать на гитхабе](https://github.com/kmike/pymorphy2))


* [документация pymorphy](https://pythonhosted.org/pymorphy/)

У него нет встроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [28]:
# основная функция - pymorphy.parse

pm_analyzed = [morph.parse(token) for token in word_tokenize(text)]

In [29]:
pm_analyzed

[[Parse(word='зрелый', tag=OpencorporaTag('ADJF,Qual masc,sing,nomn'), normal_form='зрелый', score=0.5, methods_stack=((DictionaryAnalyzer(), 'зрелый', 4, 0),)),
  Parse(word='зрелый', tag=OpencorporaTag('ADJF,Qual inan,masc,sing,accs'), normal_form='зрелый', score=0.5, methods_stack=((DictionaryAnalyzer(), 'зрелый', 4, 4),))],
 [Parse(word='фрукт', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='фрукт', score=0.3333333333333333, methods_stack=((DictionaryAnalyzer(), 'фрукт', 34, 0),)),
  Parse(word='фрукт', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='фрукт', score=0.3333333333333333, methods_stack=((DictionaryAnalyzer(), 'фрукт', 34, 3),)),
  Parse(word='фрукт', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='фрукт', score=0.3333333333333333, methods_stack=((DictionaryAnalyzer(), 'фрукт', 52, 0),))],
 [Parse(word='на', tag=OpencorporaTag('PREP'), normal_form='на', score=0.998961, methods_stack=((DictionaryAnalyzer(), 'на', 24, 0),)),
  Parse(

In [30]:
# пример с морфологической неоднозначностью

morph.parse("сахаров") 

[Parse(word='сахаров', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Surn sing,nomn'), normal_form='сахаров', score=0.5, methods_stack=((DictionaryAnalyzer(), 'сахаров', 37, 0),)),
 Parse(word='сахаров', tag=OpencorporaTag('NOUN,inan,masc plur,gent'), normal_form='сахар', score=0.5, methods_stack=((DictionaryAnalyzer(), 'сахаров', 1228, 8),))]

Она похожа на analyze в майстеме только возвращает список объектов Parse
* Первый в списке - самый вероятный разбор (у каждого есть score)
* Информация достается через атрибут (Parse.word)
* Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
части речи или другие категории

In [31]:
# сделаем красиво
print('Первое слово - ', pm_analyzed[0][0].word)
print('Разбор первого слова - ', pm_analyzed[0][0])
print('Лемма первого слова - ', pm_analyzed[0][0].normal_form)
print('Грамматическая информация первого слова - ', pm_analyzed[0][0].tag)
print('Часть речи первого слова - ', pm_analyzed[0][0].tag.POS)
print('Род первого слова - ', pm_analyzed[0][0].tag.gender)
print('Число первого слова - ',pm_analyzed[0][0].tag.number)
print('Падеж первого слова - ', pm_analyzed[0][0].tag.case)

Первое слово -  зрелый
Разбор первого слова -  Parse(word='зрелый', tag=OpencorporaTag('ADJF,Qual masc,sing,nomn'), normal_form='зрелый', score=0.5, methods_stack=((DictionaryAnalyzer(), 'зрелый', 4, 0),))
Лемма первого слова -  зрелый
Грамматическая информация первого слова -  ADJF,Qual masc,sing,nomn
Часть речи первого слова -  ADJF
Род первого слова -  masc
Число первого слова -  sing
Падеж первого слова -  nomn


### что можно дальше

Pymorphy и Mystem - не единственные морфоанализаторы для русского языка. Можно, например, посмотреть на [RNNmorph](https://github.com/IlyaGusev/rnnmorph) и [deeppavlov](http://docs.deeppavlov.ai/en/master/features/models/morphotagger.html).

А еще есть исследование, где сравнивали морфоанализаторы для русского  ([краткая версия](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/), [статья](http://www.dialog-21.ru/media/3473/dereza.pdf))

 
И на последок, морфо-анализаторы для других яззыков:
- [UralicNLP](https://github.com/mikahama/uralicNLP)
- [hfst от Apertium](https://wiki.apertium.org/wiki/Hfst)
- [Stanza](https://stanfordnlp.github.io/stanza/)
- [SpaCy](https://spacy.io/usage/linguistic-features#morphology)
- [Trankit](https://trankit.readthedocs.io/en/latest/posdep.html)